## Get the python version from the environment
#### We are using distutils.version, it is deprecated.

In [18]:
import sys
from distutils.version import LooseVersion
from packaging.version import Version, InvalidVersion

print(LooseVersion(sys.version))


3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:54:21) [Clang 16.0.6 ]


/var/folders/v1/pwt8v94j085ccrztxjd20_1m0000gn/T/ipykernel_479/2354673877.py:5: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  print(LooseVersion(sys.version))


## Compare the python version for desired version

In [10]:
if LooseVersion(sys.version) < LooseVersion('3.8'):
    print('[FAIL] We recommend Python 3.8 or newer but'
          ' found version %s' % (sys.version))
else:
    print('[OK] Your Python version is %s' % (sys.version))

[OK] Your Python version is 3.12.2 | packaged by conda-forge | (main, Feb 16 2024, 20:54:21) [Clang 16.0.6 ]


/var/folders/v1/pwt8v94j085ccrztxjd20_1m0000gn/T/ipykernel_479/3743801458.py:1: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(sys.version) < LooseVersion('3.8'):


## Check the version of a particular library

In [13]:
imported = __import__('pandas')
print(imported)
try:
    print(imported.__version__)
except AttributeError:
    try:
        print(imported.version)
    except AttributeError:
        try:
            print(imported.version_info)
        except AttributeError:
            print('0.0')



<module 'pandas' from '/opt/miniconda3/lib/python3.12/site-packages/pandas/__init__.py'>
2.2.2


In [3]:
d = {
    'numpy': '1.21.2',
    'scipy': '1.7.0',
    'matplotlib': '3.4.3',
    'sklearn': '1.0',
    'pandas': '1.3.2'
}

for p in d.keys():
    imported = __import__(p)
    print(imported)

<module 'numpy' from '/opt/miniconda3/lib/python3.12/site-packages/numpy/__init__.py'>
<module 'scipy' from '/opt/miniconda3/lib/python3.12/site-packages/scipy/__init__.py'>
<module 'matplotlib' from '/opt/miniconda3/lib/python3.12/site-packages/matplotlib/__init__.py'>
<module 'sklearn' from '/opt/miniconda3/lib/python3.12/site-packages/sklearn/__init__.py'>
<module 'pandas' from '/opt/miniconda3/lib/python3.12/site-packages/pandas/__init__.py'>


## Check python version and package versions using  packaging.version

In [20]:
from packaging.version import Version
from platform import python_version

py_ver = Version(python_version())  # e.g., "3.12.2"
print(py_ver)

3.12.2


In [22]:
from packaging.version import Version
import sys

py_ver = Version(f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}")
print(py_ver)

3.12.2


In [ ]:
from packaging.version import Version
from platform import python_version

if Version(python_version()) < Version("3.8"):
    print("[FAIL] We recommend Python >= 3.8")
else:
    print(f"[OK] Python {python_version()}")

[FAIL] We recommend Python >= 3.8


In [30]:
from importlib import import_module
from packaging.version import Version, InvalidVersion
def get_package_version_str(name: str) -> str:
    try:
        mod = import_module(name)
    except ImportError:
        return "N/A"

    for attr in ("__version__", "version", "version_info"):
        if hasattr(mod, attr):
            return str(getattr(mod, attr))
    return "0.0"

print(get_package_version_str('pandas'))
print(get_package_version_str('numpy'))

2.2.2
1.26.4


In [41]:
def parse_version(v: str) -> Version:
    try:
        return Version(str(v))
    except InvalidVersion:
        # Treat unknown/invalid versions as the lowest possible
        return Version("0")

def check_packages_with_packaging(requirements: dict[str, str]) -> None:
    # requirements: {"numpy": "1.21.2", "matplotlib": "3.4.3", ...}
    for pkg, min_ver_str in requirements.items():
        actual_str = get_package_version_str(pkg)
        if actual_str == "N/A":
            print(f"[FAIL] {pkg} is not installed.")
            continue

        actual = parse_version(actual_str)
        minimum = parse_version(min_ver_str)

        # Special-case: avoid exactly matplotlib 3.8
        if pkg == "matplotlib" and actual == Version("3.8"):
            print(f"[FAIL] {pkg} {actual}, please upgrade to > 3.8 (suggested >= {minimum})")
        elif actual < minimum:
            print(f"[FAIL] {pkg} {actual}, please upgrade to >= {minimum}")
        else:
            print(f"[OK] {pkg} {actual}")



reqs = {
        "numpy": "1.28.2",
        "scipy": "1.7.0",
        "matplotlib": "3.4.3",
        "sklearn": "1.0",
        "pandas": "1.3.2",
}
print(check_packages_with_packaging(reqs))

[FAIL] numpy 1.26.4, please upgrade to >= 1.28.2
[OK] scipy 1.13.1
[OK] matplotlib 3.9.0
[OK] sklearn 1.5.0
[OK] pandas 2.2.2
None
